## ola lyrics bot
Connect to Spotify and Genius to post lyrics on [Twitter](https://twitter.com/olalyricsbot)

In [161]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import tweepy
import random
import os
from dotenv import load_dotenv

#### Spotify API Authorization

In [162]:
load_dotenv()

auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

USERNAME = 'aleksandra.jedrych'
PLAYLIST_ID = '2VQxlEsq39DGjFxJ0o5nMo'

Download song ids from a dedicated playlist

In [163]:
def download_song_ids(username, playlist_id):
    song_ids = []
    playlist = sp.user_playlist(username, playlist_id)
    for item in playlist['tracks']['items']:
        song = item['track']
        song_ids.append(song['id'])
    return song_ids

songs_ids = download_song_ids(USERNAME, PLAYLIST_ID)
print(songs_ids)


['37ZJ0p5Jm13JPevGcx4SkF', '0GONea6G2XdnHWjNZd6zt3', '6Qyc6fS4DsZjB2mRW9DsQs', '7Ewz6bJ97vUqk5HdkvguFQ', '5t5rCnsgRBtcKqTB7SbD1Q', '3ovjw5HZZv43SxTwApooCM', '1pTGc8pwyo6xtgXBKCBcFn', '77NNZQSqzLNqh2A9JhLRkg', '5kK1Iru9ogP3Iy1zsANU1n', '5B8N5rPOmTVVGpuBMK2Vby', '60wwxj6Dd9NJlirf84wr2c', '390AWnOn2rfe9FzQjYmxIH', '6noSuYjYAXxM91DEaTAaFy', '2a3U3GTCvrcma7wetBtOpi', '1uzWOoJdADfstQuFtQFTUn', '3MrWxJaD2AT0W9DjWF64Vm', '7fpBtc7p3hFrEbDVc32ltY', '3EPXxR3ImUwfayaurPi3cm', '6N22FZs2ZhPBYi3b9XPajV', '6c9nkeetGbsoXa7d5E1wHa', '2EiGECydkS2M8OCcRHQZhT', '0eZBeB2xFIS65jQHerispi', '54b8qPFqYqIndfdxiLApea', '1PZDyestTVydtHpaIcfUJg', '7lPN2DXiMsVn7XUKtOW1CS', '4uGIJG1jYFonGc4LGp5uQL', '4vpeKl0vMGdAXpZiQB2Dtd', '39q7xibBdRboeMKUbZEB6g', '6xB7E0HOWznwiO0v56mqwD', '2qhASBzpbFhPRtrnZ5lLnz', '1zf8Xmj66XRiI6Etw2Ddu7', '0GD6Ug6ouPqkthlnT058aC', '6fqLce5TNAmNgbFwXD2DdG', '0I3cUqffwmH2mMUHIfpC0l', '5XcZRgJv3zMhTqCyESjQrF', '0bLRXQaWzmSXRXPmP6JnEF', '5lRzWDEe7UuedU2QPsFg0K', '0ow5yxh0ouMJ0om8EyEt8P', '6EkQmHBoux

Draw of a song id

In [164]:
id = songs_ids[random.randrange(len(songs_ids))]
print(id)

3iMLRCGHLQ135LtMjKLkO8


Collect data (title and main artist) needed to search forlyrics via the Genius API

In [165]:
def artist_genius(id):
    song_info = sp.track(id)
    artist = song_info['album']['artists'][0]['name'] #for genius (searching lyrics for tweet using title and main artist)
    return artist

In [166]:
artist_genius(id)

'Dionne Warwick'

In [167]:
def title(id):
    song_info = sp.track(id)
    title = song_info['name']
    return title

In [168]:
title(id)

'Heartbreaker'

Collect information about all authors to include in a tweet

In [169]:
def artists(id):
    song_info = sp.track(id)
    lista = []
    for i in range(0, len(song_info['album']['artists'])):
        lista.append(song_info['album']['artists'][i]['name']) #for tweet (including all artists performing the track)
    str_artists = ", "
    return(str_artists.join(lista))


In [170]:
artists(id)

'Dionne Warwick'

#### Connect to Genius API to download lyrics

In [171]:
genius = lyricsgenius.Genius(os.getenv("GENIUS_CLIENT_ID"))
genius.remove_section_headers = True

Download song lyrics

In [172]:
def raw_lyrics():
    lyrics = genius.search_song(title(id), artist_genius(id)).lyrics
    return lyrics

Create content of the tweet

In [173]:
def tweet_content(lyrics):
    lyrics = lyrics.split('\n')
    for index in range(len(lyrics)):
        if lyrics[index] == "" or "[" in lyrics[index]:
            lyrics[index] = "XXX"
    lyrics = [i for i in lyrics if i != "XXX"]

    random_num = random.randrange(0, len(lyrics)-1)
    tweet = lyrics[random_num] + "\n" + lyrics[random_num+1] + "\n" + "\n" + title(id) + " by " + artists(id)
    tweet = tweet.replace("\\", "")
    return tweet

#### Connect to Twitter API to post lyrics

In [174]:
api = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"), consumer_key= os.getenv("CONSUMER_KEY"), consumer_secret=os.getenv("CONSUMER_SECRET"), access_token=os.getenv("ACCESS_TOKEN"), access_token_secret=os.getenv("ACCESS_TOKEN_SECRET") )

In [175]:
api.create_tweet(text=tweet_content(lyrics=raw_lyrics()))

Searching for "Heartbreaker" by Dionne Warwick...
Done.


Response(data={'id': '1474821025120628738', 'text': 'You got me crying like I thought I would never be\nLove is believing but you let me down\n\nHeartbreaker by Dionne Warwick'}, includes={}, errors=[], meta={})